In [283]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [284]:
df = pd.read_csv("customer_feedbacks.csv")
df = df.drop(97,axis = 0)
df = df.drop(96, axis = 0)
df

,"Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score"
0,"""I love this product!"", Positive, Twitter, 202..."
1,"""The service was terrible."", Negative, Yelp Re..."
2,"""This movie is amazing!"", Positive, IMDb, 2023..."
3,"""I'm so disappointed with their customer suppo..."
4,"""Just had the best meal of my life!"", Positive..."
...,...
91,"""Just had the most amazing vacation! I can't w..."
92,"""The food at this restaurant was awful. Never ..."
93,"""I can't stop listening to this song. It's my ..."
94,"""Their website is so confusing and poorly desi..."


In [285]:
array = df['Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score'].values
data_list = array.tolist()

feedbacks = []
sentiments = []
sources = []

for i in range(96):
        feedbacks.append(str((data_list[i].split(',')[0])))
        sentiments.append(str((data_list[i].split(',')[1])))
        
        sources.append(str((data_list[i].split(',')[2])))

In [286]:
df['feedback'] = feedbacks
df['sentiment'] = sentiments
df['source'] = sources

In [287]:
df.head()


,"Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score",feedback,sentiment,source
0,"""I love this product!"", Positive, Twitter, 202...","""I love this product!""",Positive,Twitter
1,"""The service was terrible."", Negative, Yelp Re...","""The service was terrible.""",Negative,Yelp Reviews
2,"""This movie is amazing!"", Positive, IMDb, 2023...","""This movie is amazing!""",Positive,IMDb
3,"""I'm so disappointed with their customer suppo...","""I'm so disappointed with their customer suppo...",Negative,Online Forum
4,"""Just had the best meal of my life!"", Positive...","""Just had the best meal of my life!""",Positive,TripAdvisor


In [288]:
df = df.drop("Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score", axis = 1)

In [289]:
df['feedback'] = df['feedback'].str.lower()
df['sentiment'] = df['sentiment'].str.lower()
df['source'] = df['source'].str.lower()

In [290]:
df

,feedback,sentiment,source
0,"""i love this product!""",positive,twitter
1,"""the service was terrible.""",negative,yelp reviews
2,"""this movie is amazing!""",positive,imdb
3,"""i'm so disappointed with their customer suppo...",negative,online forum
4,"""just had the best meal of my life!""",positive,tripadvisor
...,...,...,...
91,"""just had the most amazing vacation! i can't w...",positive,tripadvisor
92,"""the food at this restaurant was awful. never ...",negative,zomato
93,"""i can't stop listening to this song. it's my ...",positive,spotify
94,"""their website is so confusing and poorly desi...",negative,website review


In [291]:
texts = df["feedback"]
labels = df["sentiment"]


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

texts_train, texts_test, y_train, y_test = train_test_split(texts, y, test_size=0.2, random_state=42)

In [292]:
max_len = 10 
vocab_size = 1000  
embedding_dim = 64  

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts_train)

In [293]:
def get_sequences(tokenizer, texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=max_len, padding='post')
    return padded

In [294]:
X_train = get_sequences(tokenizer, texts_train)
X_test = get_sequences(tokenizer, texts_test)

In [295]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.embedding(x)
        lstm_out, (hidden, _) = self.lstm(x)
        output = self.fc(hidden.squeeze(0))
        return output

In [296]:
hidden_dim = 128
output_dim = len(np.unique(y))
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)

lr = 0.001
num_epochs = 10
batch_size = 2
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [297]:
train_data = list(zip(torch.LongTensor(X_train), torch.LongTensor(y_train)))
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data = list(zip(torch.LongTensor(X_test), torch.LongTensor(y_test)))
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [298]:
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [301]:
import torch.nn.functional as F
model.eval()
all_preds = []
all_labels = []
for inputs, labels in test_loader:
    outputs = model(inputs)
    preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    all_preds.extend(preds.tolist())
    all_labels.extend(labels.tolist())


In [302]:
accuracy = accuracy_score(all_labels, all_preds)
print("Accuracy:", accuracy)

Accuracy: 0.85


In [303]:
torch.save(model.state_dict(), 'sentiment_lstm_model.pth')

In [304]:
import torch
import torch.nn as nn
from torch import optim
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [305]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (hidden, cell) = self.lstm(embedded)
        output = self.fc(hidden[-1])
        return output


In [306]:
max_len = 10
vocab_size = 1000
embedding_dim = 64
hidden_dim = 128
output_dim = 2  

In [307]:
import torch
import torch.nn as nn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load('sentiment_lstm_model.pth'))
model.eval()

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts_train)

max_len = 10

while True:
    user_input = input("Enter a sentence (Type 'exit' to quit): ")
    
    if user_input.lower() == 'exit':
        break
    
    tokenized = tokenizer.texts_to_sequences([user_input])
    padded = pad_sequences(tokenized, maxlen=max_len, padding='post')
    
    model_input = torch.LongTensor(padded)
    output = model(model_input)
    
    prediction = torch.argmax(output, dim=1).item()
    sentiment = label_encoder.inverse_transform([prediction])[0]
    
    print("Predicted sentiment:", sentiment)

Enter a sentence (Type 'exit' to quit):  I love this product


Predicted sentiment:  negative


Enter a sentence (Type 'exit' to quit):  this movie is amazing


Predicted sentiment:  positive


Enter a sentence (Type 'exit' to quit):  i can't stop listening to this song.


Predicted sentiment:  positive


Enter a sentence (Type 'exit' to quit):  exit
